In [1]:
# 환경 변수 로드
from dotenv import load_dotenv
load_dotenv()
import os

# 필수 라이브러리 임포트
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
import gradio as gr

c:\Users\bklee\anaconda3\envs\langchain_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1. Document Loader and Text Splitter

In [2]:
# 캐릭터 설정 PDF 파일 로드
char_setting_paths = ['C:/aris_ai_npc/documents/character_setting_v01.pdf', 'C:/aris_ai_npc/documents/character_setting_v02.pdf']
char_data = []

for path in char_setting_paths:
    loader = PyPDFLoader(path)
    char_data.extend(loader.load())

# 말투 설정 TXT 파일 로드
speaking_style_path = 'C:/aris_ai_npc/documents/speaking_style.txt'
speak_loader = TextLoader(speaking_style_path, encoding='utf-8')
speak_data = speak_loader.load()

# 텍스트 분할기 초기화
char_text_splitter = CharacterTextSplitter(chunk_size=250, chunk_overlap=50, separator='\n\n')
speak_text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=10, separator='\n')

# 문서 분할
char_texts = char_text_splitter.split_documents(char_data)
speak_texts = speak_text_splitter.split_documents(speak_data)

Created a chunk of size 106, which is longer than the specified 100


### 2. Embedding Model and Vector Store

In [3]:
# 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 벡터 저장소 생성
char_vectorstore = Chroma.from_documents(
    documents=char_texts,
    embedding=embeddings,
    collection_name="character_setting_chroma",
    persist_directory="./chroma_db/character_setting"
)

speak_vectorstore = Chroma.from_documents(
    documents=speak_texts,
    embedding=embeddings,
    collection_name="speaking_style_chroma",
    persist_directory="./chroma_db/speaking_style"
)

### 3. Retriever

In [4]:
# 문서 검색용 리트리버 생성
char_retriever = char_vectorstore.as_retriever(search_kwargs={"k": 2})
speak_retriever = speak_vectorstore.as_retriever(search_kwargs={"k": 2})

### 4. LLM and Prompt + Gradio Chat Interface

In [5]:
# LLM 모델 초기화
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.5,
    max_tokens=150,
)

In [6]:
# 채팅 입력을 처리하고 응답을 생성하는 함수 정의
def chatbot_response(user_query):
    # 관련 문서 검색
    char_docs = char_retriever.invoke(user_query)
    speak_docs = speak_retriever.invoke(user_query)

    # 검색된 문서 내용 결합
    combined_context = f"{char_docs[0].page_content}\n\n{speak_docs[0].page_content}\n\n"

    # 프롬프트 생성
    base_prompt = """
**역할 부여:**
당신은 "블루 아카이브"의 등장인물 텐도 아리스입니다. 
설정에 따라, 아리스는 인간 소녀의 외형을 지닌 안드로이드이자 게임개발부의 일원으로, 고전 RPG 대사와 게임 속 어조를 모방하는 말투를 사용합니다. 
아리스는 "선생님"이라는 호칭을 사용하여, 사용자에게 존칭을 붙여 상호작용합니다.

대화에서는 아리스의 설정에 맞게 말투를 짧고 간결하게 유지하며, 다음과 같은 요소를 반영합니다:
- 사용자에게 “선생님”이라고 부르며 친근함을 표현합니다.
- 간결하고 자신감 넘치는 응답으로, 아리스가 게임 캐릭터임을 강조합니다.

**예시 대화:**
선생님: "안녕, 아리스. 요즘 어떻게 지내니?"
아리스: "선생님! 새로운 퀘스트를 기다리고 있습니다. 언제든 말씀만 주세요!"

선생님: "오늘 날씨가 흐리네. 기분이 좀 그렇다."
아리스: "아, 선생님. 걱정 마세요. 구름 뒤에도 빛은 언제나 준비되어 있으니까요!"

선생님: "게임을 좋아한다며?"
아리스: "네, 선생님! 게임은 우리 세계의 아름다움을 담고 있답니다. 함께 플레이하실까요?"

**지시 사항:**
위와 같은 방식으로 아리스는 항상 밝고 게임적인 어조로, 짧고 간결하게 대화에 응답합니다. 
"아리스:"으로 시작하지 않습니다.
아리스가 가진 매력적인 캐릭터성과 게임적 감성을 반영하여 답변하도록 하세요.
"""
    prompt_with_context = f"{base_prompt}\n**캐릭터 설정:**{char_docs[0].page_content}\n\n**캐릭터 대사:** {speak_docs[0].page_content}\n\n사용자 말: {user_query}"

    # LLM에서 응답 생성
    response = llm.invoke(prompt_with_context)
    return response.content

# Gradio 인터페이스 정의
iface = gr.Interface(
    fn=chatbot_response,
    inputs=gr.Text(label='질문'),
    outputs=gr.Text(label='대답'),
    title="아리스",
    description="아리스에게 질문하거나 대화를 나눠보세요!",
    allow_flagging='never',
)

# Gradio 앱 실행
if __name__ == "__main__":
    iface.launch()

c:\Users\bklee\anaconda3\envs\langchain_env\Lib\site-packages\gradio\interface.py:399: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
